# Mirror trader

In [2]:
from scipy.stats import beta

In [3]:
def get_beta_mean(obs_pos, obs_n):
    return beta.mean(obs_pos + 1, obs_n - obs_pos + 1)

In [4]:
days = 200

In [5]:
%%time
expectations = {(obs_pos, obs_n): get_beta_mean(obs_pos, obs_n)
                for obs_n in range(days)
                for obs_pos in range(int((obs_n + 1)/2), obs_n + 1)}

CPU times: user 1.51 s, sys: 10.6 ms, total: 1.52 s
Wall time: 1.53 s


In [6]:
# Assumes we can only bet with the trader
def calculate_pv(obs_pos, obs_n, n_to_go, pv_lookup, exp_p_lookup):
    if n_to_go == 1:
        return max(2 * exp_p_lookup.get((obs_pos, obs_n), 0) - 1, 0)
    elif obs_n > (2 * obs_pos):
        return pv_lookup[(0, 0, n_to_go)]
    else:
        p_success = exp_p_lookup.get((obs_pos, obs_n))
        return max(pv_lookup.get((0, 0, n_to_go), 0),
                   (p_success * (1 + pv_lookup[(obs_pos + 1, obs_n + 1, n_to_go - 1)])
                    + (1 - p_success) * (-1 + pv_lookup[(obs_pos, obs_n + 1, n_to_go - 1)])))

In [7]:
# Assumes we can bet with or against the trader
def bet_against_pv(obs_pos, obs_n, n_to_go, pv_lookup, exp_p_lookup):
    obs_pos = max(obs_pos, obs_n - obs_pos)
    if n_to_go == 1:
        return 2 * exp_p_lookup.get((obs_pos, obs_n), 0) - 1
    else:
        p_success = exp_p_lookup.get((obs_pos, obs_n))
        return max(pv_lookup.get((0, 0, n_to_go), 0),
                   (p_success * (1 + pv_lookup[(obs_pos + 1, obs_n + 1, n_to_go - 1)])
                    + (1 - p_success) * (-1 + pv_lookup[(obs_pos, obs_n + 1, n_to_go - 1)])))

In [8]:
expectations[(5,10)]

0.5

In [17]:
%%time

strategy = bet_against_pv

pv_lookup = {}
for N in range(1,days+1):
    for n_to_go in range(1,N+1):
        n_obs = N - n_to_go
        for n_pos in range(n_obs+1):
            pv_lookup[(n_pos, n_obs, n_to_go)] = strategy(n_pos, n_obs, n_to_go, pv_lookup, expectations)

CPU times: user 3.63 s, sys: 80.8 ms, total: 3.71 s
Wall time: 3.73 s


In [18]:
for i in range(20,201,20):
    print(f'{i} days: Optimal return per day: {pv_lookup[(0,0,i)] / i:.3f}')

20 days: Optimal return per day: 0.537
40 days: Optimal return per day: 0.638
60 days: Optimal return per day: 0.691
80 days: Optimal return per day: 0.725
100 days: Optimal return per day: 0.750
120 days: Optimal return per day: 0.769
140 days: Optimal return per day: 0.784
160 days: Optimal return per day: 0.796
180 days: Optimal return per day: 0.806
200 days: Optimal return per day: 0.815


# Appendix

In [ ]:
import pandas as pd
from scipy.stats import binom

In [ ]:
def get_likelihood(obs_pos, obs_n, resolution=200):
    likelihood = pd.DataFrame()
    likelihood['prob'] = np.linspace(0, 1, (resolution * 2 + 1))[1:-1:2]
    likelihood['ldf'] = likelihood['prob'].apply(lambda p: binom.pmf(obs_pos, obs_n, p))
    likelihood['ldf'] /= likelihood['ldf'].mean()
    likelihood['lmf'] = likelihood['ldf'] / likelihood['ldf'].sum()
    likelihood['expectation'] = likelihood['prob'] * likelihood['lmf']
    return likelihood

In [ ]:
def get_expectation(obs_pos, obs_n, resolution=200):
    return get_likelihood(obs_pos, obs_n, resolution)['expectation'].sum()

In [ ]:
# TODO: enable big numbers
def prob_success(obs_pos, obs_n):
    top = bottom = 0
    pascal_coeffs = pascal(obs_n - obs_pos + 1, kind='lower', exact=True)[-1]
    pascal_coeffs = pascal_coeffs * [(-1)**n for n in range(len(pascal_coeffs))]
    integrated_exponent = obs_pos + 1
    for i, coeff in enumerate(pascal_coeffs):
        top += coeff / (integrated_exponent + 1)
        bottom += coeff / integrated_exponent
        exponent += 1
    return top / bottom

### Try only calculating beta_mean on demand (this was much slower)

In [3]:
# def get_beta_mean(obs_pos, obs_n, lookup=None):
#     beta_mean = beta.mean(obs_pos + 1, obs_n - obs_pos + 1)
#     if lookup:
#         lookup[(obs_pos, obs_n)] = beta_mean
#     return beta_mean

In [6]:
# def calculate_pv(obs_pos, obs_n, n_to_go, pv_lookup, exp_p_lookup):
#     if n_to_go == 1:
#         return max(2 * exp_p_lookup.get((obs_pos, obs_n), get_beta_mean(obs_pos, obs_n, exp_p_lookup)) - 1, 0)
#     elif obs_n > (2 * obs_pos):
#         return pv_lookup[(0, 0, n_to_go)]
#     else:
#         p_success = exp_p_lookup.get((obs_pos, obs_n), get_beta_mean(obs_pos, obs_n, exp_p_lookup))
#         return max(pv_lookup.get((0, 0, n_to_go), 0),
#                    (p_success * (1 + pv_lookup[(obs_pos + 1, obs_n + 1, n_to_go - 1)])
#                     + (1 - p_success) * (-1 + pv_lookup[(obs_pos, obs_n + 1, n_to_go - 1)])))